In [1]:
%pip install langchain-experimental


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install --upgrade langchain-core langchain-community
import os
import re
from copy import deepcopy
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from typing import List, Dict, Any
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import networkx as nx
# from sentence_transformers import SentenceTransformer
import tiktoken


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
def load_pdf(pdf_path: str) -> List[Document]:
    try:
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        
        for i, doc in enumerate(documents):
            print(f"Page {i+1}: {len(doc.page_content)} characters, Metadata: {doc.metadata}")
            
        return documents
    except Exception as e: 
        print(f"Error loading PDF: {e}")
        return []
    
pdf_path = '/home/jeremy/Documents/Work/Learning/fastapi/llm/4-document_loader/sample_data/document.pdf'
documents = load_pdf(pdf_path)
print(documents)

Page 1: 1865 characters, Metadata: {'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Copy of 251102 PromptBI Analysts Training Agreement Draft v.2', 'source': '/home/jeremy/Documents/Work/Learning/fastapi/llm/4-document_loader/sample_data/document.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}
Page 2: 3041 characters, Metadata: {'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Copy of 251102 PromptBI Analysts Training Agreement Draft v.2', 'source': '/home/jeremy/Documents/Work/Learning/fastapi/llm/4-document_loader/sample_data/document.pdf', 'total_pages': 4, 'page': 1, 'page_label': '2'}
Page 3: 1750 characters, Metadata: {'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Copy of 251102 PromptBI Analysts Training Agreement Draft v.2', 'source': '/home/jeremy/Documents/Work/Learning/fastapi/llm/4-document_loader/sample_data/docu

In [ ]:
from collections import defaultdict

def topic_aware_text_splitting(
    documents: List[Document],
    max_chunk_size: int = 1000,
    similarity_threshold: float = .7
) -> List[Document]:
    try:
        from sklearn.cluster import DBSCAN
        from sentence_transformers import SentenceTransformer
    except ImportError:
        raise ImportError("Please install sklearn and sentence_transformers packages")
    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    all_chunks = []
    
    for doc in documents:
        text = doc.page_content
        sentences = sent_tokenize(text)
        # create embeddings for sentences
        embeddings = model.encode(sentences)
        # compute cosine similarity matrix
        sim_matrix = cosine_similarity(embeddings)
        # clustering sentences
        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=1 - similarity_threshold,
            affinity="precomputed",
            linkage="average"
        )
        
        labels = clustering.fit_predict(1 - sim_matrix)
        
        # group sentences by cluster labels
        clustered_sentences = defaultdict(list)
        for label, sentence in zip(labels, sentences):
            clustered_sentences[label].append(sentence)
            
        # create chunks
        for cluster in clustered_sentences.values():
            chunk_text = ' '.join(cluster)
            if len(chunk_text) > max_chunk_size:
                sub_chunks = re.findall(r".{1,}" + str(max_chunk_size) + r'}(?:\s|$)', chunk_text)
                for sub_chunk in sub_chunks:
                    all_chunks.append(Document(
                        page_content=sub_chunk.strip(),
                        metadata=doc.metadata
                    ))
            else: 
                all_chunks.append(Document(
                    page_content=chunk_text.strip(),
                    metadata=doc.metadata
                ))
                
    return all_chunks

topic_chunks = topic_aware_text_splitting(documents, max_chunk_size=700, similarity_threshold=.7)
print(topic_chunks)

In [ ]:
from sentence_transformers import SentenceTransformer


# -----------------------
# CONFIG
# -----------------------

MODEL_NAME = "all-MiniLM-L6-v2"
MAX_TOKENS = 350
OVERLAP_SENTENCES = 1

CLAUSE_RE = re.compile(
    r"(Section|Clause|Article)\s+\d+(\.\d+)*(\([a-z]\))*",
    re.I
)

enc = tiktoken.get_encoding("cl100k_base")
tok_len = lambda t: len(enc.encode(t))


# -----------------------
# HELPERS
# -----------------------

def extract_clauses(text):
    return ["".join(m) for m in CLAUSE_RE.findall(text)]


def find_offset(full, chunk, cursor):
    pos = full.find(chunk, cursor)
    return pos if pos != -1 else cursor


# -----------------------
# MAIN
# -----------------------

def topic_aware_text_splitting(
    documents: List[Document],
    similarity_threshold: float = 0.70,
) -> List[Document]:

    model = SentenceTransformer(MODEL_NAME)
    all_chunks = []

    for doc_idx, doc in enumerate(documents):

        text = doc.page_content
        sentences = sent_tokenize(text)

        if len(sentences) < 3:
            all_chunks.append(doc)
            continue

        embeds = model.encode(sentences, normalize_embeddings=True)
        sim = cosine_similarity(embeds)

        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=1 - similarity_threshold,
            metric="precomputed",
            linkage="average"
        )

        labels = clustering.fit_predict(1 - sim)

        # ---- group while preserving order ----
        groups = defaultdict(list)
        for i, label in enumerate(labels):
            groups[label].append((i, sentences[i]))

        ordered_groups = sorted(groups.values(), key=lambda g: g[0][0])

        cursor = 0
        chunk_idx = 0

        # ---- build token-bounded chunks ----
        for group in ordered_groups:

            ordered_sents = [s for _, s in group]

            buf = []
            buf_tokens = 0

            for sent in ordered_sents:

                t = tok_len(sent)

                if buf_tokens + t > MAX_TOKENS and buf:

                    chunk_text = " ".join(buf)
                    start = find_offset(text, chunk_text, cursor)
                    end = start + len(chunk_text)

                    meta = deepcopy(doc.metadata or {})
                    meta.update({
                        "chunk_id": f"{doc_idx}_{chunk_idx}",
                        "start_char": start,
                        "end_char": end,
                        "token_count": tok_len(chunk_text),
                        "topic_cluster": int(group[0][0]),
                        "clauses": extract_clauses(chunk_text)
                    })

                    all_chunks.append(Document(
                        page_content=chunk_text,
                        metadata=meta
                    ))

                    cursor = end
                    chunk_idx += 1

                    # overlap
                    buf = buf[-OVERLAP_SENTENCES:]
                    buf_tokens = tok_len(" ".join(buf))

                buf.append(sent)
                buf_tokens += t

            if buf:
                chunk_text = " ".join(buf)
                start = find_offset(text, chunk_text, cursor)
                end = start + len(chunk_text)

                meta = deepcopy(doc.metadata or {})
                meta.update({
                    "chunk_id": f"{doc_idx}_{chunk_idx}",
                    "start_char": start,
                    "end_char": end,
                    "token_count": tok_len(chunk_text),
                    "clauses": extract_clauses(chunk_text)
                })

                all_chunks.append(Document(
                    page_content=chunk_text,
                    metadata=meta
                ))

                cursor = end
                chunk_idx += 1

    return all_chunks
